In [1]:
import sys
sys.path.append('..')

In [12]:
import sentence_embedding_evaluation_german as seeg
import torch
from typing import List

## (1) Instantiate your Embedding model
First, you should load your pretrained embedding.

Here we will generate a random embedding for demonstration purposes.

In [3]:
# generate a random embedding
emb_dim = 512
vocab_sz = 128
emb = torch.randn((vocab_sz, emb_dim), requires_grad=False)
emb = torch.nn.Embedding.from_pretrained(emb)
# assert emb.weight.requires_grad == False

## (2) Specify the preprocessing
The `preprocessor` function converts a sentences as string into embedding vectors of numbers.

Here we will convert the input strings with a nonsensical approach into IDs for the Embedding layer.

In [4]:
def preprocesser(batch: List[str], params: dict=None) -> List[List[float]]:
    """ Specify your embedding or pretrained encoder here
    Paramters:
    ----------
    params : dict
        The params dictionary
    batch : List[str]
        A list of sentence as string
    Returns:
    --------
    List[List[float]]
        A list of embedding vectors
    """
    features = []
    for sent in batch:
        try:
            ids = torch.tensor([ord(c) % 128 for c in sent])
        except:
            print(sent)
        h = emb(ids)
        features.append(h.mean(axis=0))
    features = torch.stack(features, dim=0)
    return features

## (3) Training settings

In [5]:
params = {
    'datafolder': '../datasets',
    'batch_size': 64, 
    'num_epochs': 20,
    # 'early_stopping': True,
    # 'split_ratio': 0.2,  # if early_stopping=True
    # 'patience': 5,  # if early_stopping=True
}

## (4) Specify downstream tasks

In [6]:
# downstream_tasks = [
#     'TOXIC', 'ENGAGE', 'FCLAIM', 
#     'OL19A', 'OL19B', 'OL19C',
#     'OL18A', 'OL18B', 
#     'ABSD1', 'ABSD2', 'ABSD3']

downstream_tasks = [
    'TOXIC', 'ENGAGE', 'FCLAIM', 
    'OL19B', 'OL19C',
    'ABSD1', 'ABSD2']

## (5) Run experiments

In [7]:
%%time
results = seeg.evaluate(downstream_tasks, preprocesser, **params)

CPU times: user 42.5 s, sys: 5.23 s, total: 47.8 s
Wall time: 46 s


## (6) Display results

In [8]:
[(res['task'], res['epochs'], res['train']['num'], res['test']['num']) for res in results]

[('TOXIC', 20, 3244, 944),
 ('ENGAGE', 20, 3244, 944),
 ('FCLAIM', 20, 3244, 944),
 ('OL19B', 20, 3979, 3030),
 ('OL19C', 20, 1920, 929),
 ('ABSD1', 20, 19431, 2554),
 ('ABSD2', 20, 19431, 2554)]

In [9]:
metric = 'f1'
mode = 'train'

[(res['task'], res[mode][metric]) for res in results]

[('TOXIC', 0.6683107274969173),
 ('ENGAGE', 0.7361282367447596),
 ('FCLAIM', 0.6658446362515413),
 ('OL19B', 0.678059814023624),
 ('OL19C', 0.8661458333333333),
 ('ABSD1', 0.8337193144974525),
 ('ABSD2', 0.6797385620915033)]

In [10]:
metric = 'f1'
mode = 'test'

[(res['task'], res[mode][metric]) for res in results]

[('TOXIC', 0.6271186440677966),
 ('ENGAGE', 0.736228813559322),
 ('FCLAIM', 0.6684322033898306),
 ('OL19B', 0.6801980198019802),
 ('OL19C', 0.8557588805166847),
 ('ABSD1', 0.8198903680501175),
 ('ABSD2', 0.6534847298355521)]

In [11]:
metric = 'f1-balanced'
mode = 'test'

[(res['task'], res[mode][metric]) for res in results]

[('TOXIC', 0.38809213385484564),
 ('ENGAGE', 0.4392306646277021),
 ('FCLAIM', 0.4096593281666137),
 ('OL19B', 0.2024160282852092),
 ('OL19C', 0.4611368909512761),
 ('ABSD1', 0.45051635111876076),
 ('ABSD2', 0.26347778040887204)]